In [3]:
import requests
from bs4 import BeautifulSoup

In [9]:
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cms.gov", None)

MyAppToken = '5YNrrmFobthjbfVSPQWEZWkkk'
username = 'jennifer.faith16@gmail.com'
password = 'Nashua1998!'

#Example authenticated client (needed for non-public datasets):
client = Socrata("data.cms.gov",
                 MyAppToken,
                 username,
                 password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("s2uc-8wxp", limit = 10)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [11]:
results

[{'week_ending': '2020-05-24T00:00:00.000',
  'federal_provider_number': '045143',
  'provider_name': 'CONCORDIA NURSING & REHAB, LLC',
  'provider_address': '7 PROFESSIONAL DRIVE',
  'provider_city': 'BELLA VISTA',
  'provider_state': 'AR',
  'provider_zip_code': '72714',
  'submitted_data': 'N',
  'county': 'Benton',
  'geolocation': {'type': 'Point',
   'coordinates': [-94.25299200000002, 36.474911]},
  'reporting_interval': 'Week 01 - May 24',
  ':@computed_region_dz63_q6gx': '53',
  ':@computed_region_ia25_mrsk': '85',
  ':@computed_region_p3pw_njwh': '15',
  ':@computed_region_x9pa_s2gq': '948',
  ':@computed_region_vbss_8z7g': '32607'},
 {'week_ending': '2020-05-24T00:00:00.000',
  'federal_provider_number': '045189',
  'provider_name': 'SOMERSET SENIOR LIVING AT PINE HILLS',
  'provider_address': '900 MAGNOLIA RD',
  'provider_city': 'CAMDEN',
  'provider_state': 'AR',
  'provider_zip_code': '71701',
  'submitted_data': 'N',
  'county': 'Ouachita',
  'geolocation': {'type': 'Po

In [16]:
data=[]
for i in range(len(response)):  
    data.append({
        "week_ending": response[i].get('week_ending'),
        "federal_provider_number": response[i].get('federal_provider_number')
    })

In [4]:
import json

In [5]:
import pandas as pd

In [1]:
data = 'https://data.cms.gov/resource/s2uc-8wxp.json'

In [6]:
request = requests.get(data)

In [7]:
print(request.status_code)

200


In [8]:
response = request.json()

In [15]:
response[0].get('week_ending')

'2020-05-24T00:00:00.000'

In [17]:
len(data)

1000

In [18]:
df=pd.DataFrame(data)
df.head()

,week_ending,federal_provider_number
0,2020-05-24T00:00:00.000,045143
1,2020-05-24T00:00:00.000,045189
2,2020-05-24T00:00:00.000,045451
3,2020-05-24T00:00:00.000,055474
4,2020-05-31T00:00:00.000,055527


In [19]:
df['week_ending'].nunique()

38

In [22]:
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cms.gov", None)

MyAppToken = '5YNrrmFobthjbfVSPQWEZWkkk'
username = 'jennifer.faith16@gmail.com'
password = 'Nashua1998!'

#Example authenticated client (needed for non-public datasets):
client = Socrata("data.cms.gov",
                 MyAppToken,
                 username,
                 password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("s2uc-8wxp", limit = 1000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

KeyboardInterrupt: 

In [21]:
results_df.shape

(1000, 125)

In [ ]:
results_df.head()